In [2]:
from models.llama import *
llama_model, llama_tokenizer = get_model_and_tokenizer("7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
good_tokens_str = list("0123456789, ")
good_tokens = [llama_tokenizer.convert_tokens_to_ids(st) for st in good_tokens_str]
# good_tokens = llama_tokenizer.encode(good_tokens_str)
good_tokens += [llama_tokenizer.eos_token_id]
bad_tokens = [i for i in range(len(llama_tokenizer)) if i not in good_tokens]
print("Good Tokens:")
print("token | string")
for token in good_tokens:
    # print(f"{token} | {llama_tokenizer.convert_ids_to_tokens(token)}")
    print(f"{token} | {llama_tokenizer.decode(token)}")

Good Tokens:
token | string
29900 | 0
29896 | 1
29906 | 2
29941 | 3
29946 | 4
29945 | 5
29953 | 6
29955 | 7
29947 | 8
29929 | 9
29892 | ,
0 | <unk>
2 | </s>


In [160]:
input_str = "243,665,346,645,"
# input_str = "It was the best of times it was the worst of"

batch = llama_tokenizer([input_str], return_tensors="pt")

print("Input Tokens:")
print("token | string")
for token in batch["input_ids"].squeeze():
    # print(f"{token} | {llama_tokenizer.convert_ids_to_tokens(int(token))}")
    print(f"{token} | {llama_tokenizer.decode(token)}")

Input Tokens:
token | string
1 | <s>
29871 | 
29906 | 2
29946 | 4
29941 | 3
29892 | ,
29953 | 6
29953 | 6
29945 | 5
29892 | ,
29941 | 3
29946 | 4
29953 | 6
29892 | ,
29953 | 6
29946 | 4
29945 | 5
29892 | ,


In [161]:
generate_ids = llama_model.generate(
            **batch,
            do_sample=True,
            # max_new_tokens=max_tokens,
            max_new_tokens=30,
            temperature=0.9, 
            top_p=0.9, 
            # bad_words_ids=[[t] for t in bad_tokens],
            renormalize_logits=True,
        )
gen_strs = llama_tokenizer.batch_decode(
            generate_ids, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False
        )
gen_strs = [x.replace(input_str, '').strip() for x in gen_strs]
print(gen_strs)

/home/admin-quad/anaconda3/envs/LLM/lib/python3.11/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


['321,558,427,071,168,527,356,49']


In [165]:
batch = llama_tokenizer([input_str], return_tensors="pt",add_special_tokens=True)
batch = {k: v.cuda() for k, v in batch.items()}
with torch.no_grad():
        out = llama_model(**batch)

In [170]:
out['logits'].shape

torch.Size([1, 18, 32000])

### GPT Tokenzier

In [146]:
from models.gpt import *
import tiktoken

In [158]:
# model = 'text-davinci-003'
model = 'gpt-3.5-turbo'
# model = 'gpt-4'

# input_str = "243,665,346,645,"
# input_str = "2 4 3 , 6 6 5 , 3 4 6 , 6 4 5 ,"
# input_str = "243,665,346,645,645346"
input_str = "42235630"
# input_str = "100,200,300,400,"
# input_str = "0,1,2,3,4,5"


# input_str = "It was the best of times it was the worst of"

In [159]:
allowed_tokens = [str(i) for i in range(10)]
allowed_tokens += [" ,", " ", "", ' -'] 
get_allowed_ids(allowed_tokens, model)


encoder = tiktoken.encoding_for_model(model)
tokens = encoder.encode(input_str)
print("token | string")
for token in tokens:
    print(f"{token} | {encoder.decode([token])}")

token | string
16460 | 422
18349 | 356
966 | 30


### Explore Serializer

In [ ]:
from data.serialize import *

In [ ]:
settings=SerializerSettings(base=10, prec=3, signed=True, time_sep=',', bit_sep='')
serialize_arr(np.array([112.15379,117.89963,131.78541,128.91249]), settings)

In [4]:
torch.nn.functional.log_softmax

<function torch.nn.functional.log_softmax(input: torch.Tensor, dim: Optional[int] = None, _stacklevel: int = 3, dtype: Optional[int] = None) -> torch.Tensor>